# Infer network

Infer a network and plot with gene labels.

## More details
Plot networks inferred from datasets simulated from realistic network structures. Several datasets are available, from different organisms, different numbers of genes in the networks and different numbers of cells in the dataset. To infer a network from a different dataset, make sure the data file is in the same format as the simulated datasets, and change `dataset_name` to the relevant path.

For very large datasets (tens of thousands of cells, thousands of genes), network inference may take a long time. Benchmarking scripts are included to indicate how number of cells, number of genes and algorithm affect the time taken to infer a network.

The network inference algorithms rank all edges, between every possible pair of genes. In order to progress from a ranked list to a network, a threshold must be set, indicating what percentage of the highest-ranked edges to include.

In [ ]:
# Include packages

using NetworkInference
using Junet

include("../helper_functions.jl")

In [ ]:
# Customize the dataset, algorithm and percentage threshold (for plotting)

# 50 or 100
number_of_genes = 50
# "ecoli1", "ecoli2", "yeast1", "yeast2" or "yeast3"
organism = "yeast1"
# "large", "medium", or "small"
size = "large"

# Or change to custom path
dataset_name = string("../simulated_datasets/", number_of_genes, "_", organism, "_", size, ".txt")

# PIDCNetworkInference() or MINetworkInference()
algorithm = PIDCNetworkInference()

# 0.0 < threshold < 1.0
threshold = 0.15

In [ ]:
# Get the genes and discretize the expression levels

@time genes = get_genes(dataset_name);

In [ ]:
# Infer the network

@time network = NetworkAnalysis(algorithm, genes);

In [ ]:
# Plot the network at the given threshold

plot(network_analysis_to_graph(network, threshold))